# Problemática

Com a base de dados disponíveis devemos calcular o risco de inadimplência (default), onde devemos considerar
os clientes defaults como aqueles com mais de 60 dias de atraso, onde realizaremos a marcação desses clientes da seguinte forma:

 - **Maus** Pagadores: Aqueles que entraram em default (60 dias de atraso ou mais) nos 24 meses seguintes a aquisição do cartão de crédito
 
 - **Bons** Pagadores: São considerados todos os demais
  
 - **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa ou desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus

# Bases disponíveis

Temos 2 bases importantes para trabalharmos, sendo elas:

 - **Propostas**: Com diversas informações dos vários solicitantes de cartão de crédito
 
 - **Base de Pagamentos**: Esta base será utilizada para identificar a ocorrência de default.
 
 A base de propostas tem diversas informações coletadas no momento de solicitação de crédito
 
 As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

# Importar bibliotecas

In [1]:
import pandas as pd

# Leitura dos dados

In [2]:
filename_propostas = './credit_scoring/application_record.csv'
filename_pg = './credit_scoring/pagamentos_largo.csv'
propostas = pd.read_csv(filename_propostas)
pg = pd.read_csv(filename_pg)

# Exibir todas as colunas

In [3]:
pd.set_option('max_columns',None)

# Visualização dos dados

In [4]:
propostas.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [5]:
pg.head(5)

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,NaN,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,C,C,C,C


# Coletar o total de defaults por coluna

In [6]:
pg['referencia'] = pg.isin(['2', '3', '4', '5']).sum()
pg.head(5)

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,referencia
0,5001718,0,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,NaN,NaN,0,NaN,0,0,0,0,NaN,0,NaN,NaN
1,5001719,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,NaN
2,5001720,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,NaN
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,C,C,C,C,NaN


# Marcar todos os clientes que possuem histórico de default

In [7]:
# Exibir os defaults por linha
pg.isin(['2', '3', '4', '5']).sum(axis=1)

0        0
1        0
2        0
3        0
4        0
        ..
20932    0
20933    0
20934    0
20935    0
20936    0
Length: 20937, dtype: int64

In [8]:
# Agregar dados em uma coluna para referência
pg['referencia'] = pg.isin(['2', '3', '4', '5']).sum(axis=1)
pg.head(5)

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,referencia
0,5001718,0,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,NaN,NaN,0,NaN,0,0,0,0,NaN,0,NaN,0
1,5001719,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,0
2,5001720,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,0
4,5001726,0,0,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,C,C,C,C,0


In [9]:
# Criar uma flag para mapear todos os clientes
pg['fl_default'] = 0

In [10]:
# Mapear todos os clientes com histórico de default
pg.loc[pg['referencia'] != 0, 'fl_default'] = 1
pg.head(5)

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,referencia,fl_default
0,5001718,0,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,NaN,NaN,0,NaN,0,0,0,0,NaN,0,NaN,0,0
1,5001719,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,0,0
2,5001720,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,0,0
4,5001726,0,0,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,C,C,C,C,0,0


In [11]:
# Exibir clientes com histórico de default ao longo dos 24 meses
pg.loc[pg['fl_default'] == 1]

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,referencia,fl_default
130,5002126,0,0,C,C,C,C,C,C,C,C,C,C,NaN,C,C,C,C,C,NaN,1,2,2,3,3,2,5,1
177,5002283,0,0,1,2,0,0,0,0,0,0,0,C,0,C,C,C,C,C,0,0,0,0,0,0,0,1,1
303,5002642,0,0,0,0,1,0,0,0,C,C,C,C,0,C,C,C,C,C,0,0,0,0,0,1,2,1,1
355,5002795,0,1,0,0,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,0,1,2,0,0,1,1
556,5003267,1,2,C,C,C,C,C,C,C,C,C,C,3,C,C,C,C,C,3,1,1,C,C,C,C,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20523,5149192,0,0,5,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,5,5,5,5,5,5,5,11,1
20734,5149698,0,0,NaN,NaN,NaN,4,2,2,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,0,1,NaN,2,0,4,1
20744,5149728,0,0,0,0,2,C,C,C,C,C,C,C,0,C,C,C,NaN,NaN,0,0,0,0,0,0,0,1,1
20745,5149730,0,0,0,0,2,C,C,C,C,C,C,C,0,C,C,C,NaN,NaN,0,0,0,0,0,0,0,1,1


# Localizar os clientes da base de propostas que se encontram na base de pagamentos

In [12]:
propostas.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [19]:
# Mapear os clientes que constam nos 2 dataframes
propostas['referencia'] = propostas['ID'].isin(pg['ID'])
propostas.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,referencia
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,False
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,False
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,True
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,False
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,False


In [21]:
# Listar os clientes que aceitaram a proposta
cl_ac_propostas = propostas.loc[propostas['referencia'] == True]
cl_ac_propostas.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,referencia
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,True
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,True
6,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,True
11,5112956,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.0,True
19,5008825,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0,True


# Consolidar informações em um único dataframe

In [30]:
# Unificar dataframes
df = cl_ac_propostas.merge(pg, how='left', on='ID')
df.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,referencia_x,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,referencia_y,fl_default
0,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,True,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0,NaN,C,C,C,NaN,0,NaN,NaN,NaN,0,NaN,0,0
1,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,True,0,NaN,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,NaN,0,NaN,0,NaN,0,0
2,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,True,0,NaN,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,NaN,0,NaN,0,NaN,0,0
3,5112956,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.0,True,0,0,0,0,NaN,0,0,0,0,NaN,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,NaN,0,0
4,5008825,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [31]:
# Tratar dados do dataframe
df.drop(columns=['referencia_x', 'referencia_y'], inplace=True)

In [32]:
df.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,mes_18,mes_19,mes_2,mes_20,mes_21,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,fl_default
0,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0,NaN,C,C,C,NaN,0,NaN,NaN,NaN,0,NaN,0
1,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0,NaN,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,NaN,0,NaN,0,NaN,0
2,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0,NaN,C,C,C,C,C,C,C,C,C,C,0,C,C,C,C,C,0,0,NaN,0,NaN,0,NaN,0
3,5112956,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.0,0,0,0,0,NaN,0,0,0,0,NaN,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,NaN,0
4,5008825,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [33]:
# Preencher os dados faltantes
df.fillna('-', inplace=True)

In [34]:
# Contabilizar os defaults do dataframe
df['fl_default'].value_counts()

0    16260
1      390
Name: fl_default, dtype: int64